# Create a PortMirror service to snoop on dataplane traffic with your slice

FABRIC provides a special PortMirror Network Service that allows experimenters to snoop on traffic within a the slice. This service requires a __Net.PortMirror__ project permission tag on the project and granting this permission requires a review by the FABRIC team to ensure the project takes proper security precautions to prevent the misuse of this feature. 

In this notebook we demonstrate the process by creating a slice with a FABNetv4 service and PortMirror service where traffic from FABNetv4 service is mirrored. 

## Step 0: Import the FABlib Library

In [1]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

Orchestrator,beta-7.fabric-testbed.net
Credential Manager,beta-2.fabric-testbed.net
Core API,beta-3.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,b9847fa1-13ef-49f9-9e07-ae6ad06cda3f
Bastion Host,bastion.fabric-testbed.net
Bastion Username,kthare10_0011904101
Bastion Private Key File,/home/fabric/work/fabric_config/bastion-prod-2
Slice Public Key File,/home/fabric/work/fabric_config/id_rsa.pub
Slice Private Key File,/home/fabric/work/fabric_config/id_rsa
Sites to avoid,


## Step 1: Create the slice that generates traffic and also monitors traffic
We will use the same approach as a [FABNetv4 full-auto notebook](../create_l3network_fabnet_ipv4_auto/create_l3network_fabnet_ipv4_auto.ipynb) here. 

### Initialize variables
The following code is split up into two cells so they can be executed separately, if needed.

In [8]:
slice_name = 'Traffic Generator_Listener Slice'

[site1, site2] = fablib.get_random_sites(2)
# you can use the below line to override site locations
#site1 = ('CLEM')

print(f'Will create "{slice_name}" on {site1} and {site2}')

node1_name = 'Node1'
node2_name = 'Node2'

network1_name='net1'
network2_name='net2'

listener_node_name = 'listener_node'
listener_pm_service = 'pmservice'
listener_direction = 'both' # can also be 'rx' and 'tx'

Will create "Traffic Generator_Listener Slice" on RENC and UKY


### Create the slice

In [10]:
# Create Traffic generator slice

slice = fablib.new_slice(name=slice_name)

# Networks
net1 = slice.add_l3network(name=network1_name, type='IPv4')
net2 = slice.add_l3network(name=network2_name, type='IPv4')

# Node1
node1 = slice.add_node(name=node1_name, site=site1)
iface1 = node1.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface1.set_mode('auto')
net1.add_interface(iface1)
node1.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net1.get_gateway())

# Node2
node2 = slice.add_node(name=node2_name, site=site2)
iface2  = node2.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface2.set_mode('auto')
net2.add_interface(iface2)
node2.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net2.get_gateway())


#Submit Slice Request
slice.submit();


Retry: 9, Time: 312 sec


ID,b3c38986-639c-42e3-860a-35d3fc3fa594
Name,Traffic Generator_Listener Slice
Lease Expiration (UTC),2024-07-31 21:19:22 +0000
Lease Start (UTC),2024-07-30 21:19:22 +0000
Project ID,b9847fa1-13ef-49f9-9e07-ae6ad06cda3f
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
2e640c4f-d444-41a8-85e8-ace38ce17015,Node1,2,8,10,default_rocky_8,qcow2,renc-w1.fabric-testbed.net,RENC,rocky,152.54.15.39,Active,,ssh -i /home/fabric/work/fabric_config/id_rsa -F /home/fabric/work/fabric_config/ssh_config rocky@152.54.15.39,/home/fabric/work/fabric_config/id_rsa.pub,/home/fabric/work/fabric_config/id_rsa
8360385c-c21c-4f34-a27d-3c77d6aac2f1,Node2,2,8,10,default_rocky_8,qcow2,uky-w1.fabric-testbed.net,UKY,rocky,128.163.179.49,Active,,ssh -i /home/fabric/work/fabric_config/id_rsa -F /home/fabric/work/fabric_config/ssh_config rocky@128.163.179.49,/home/fabric/work/fabric_config/id_rsa.pub,/home/fabric/work/fabric_config/id_rsa


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
505b620d-b7d1-4b47-ba4e-15d26ae5787c,net1,L3,FABNetv4,RENC,10.128.1.0/24,10.128.1.1,Active,
acecd0b0-5b68-409c-9579-f9674becb657,net2,L3,FABNetv4,UKY,10.128.129.0/24,10.128.129.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-nic1-p1,p1,Node1,net1,100,auto,,02:37:BE:04:DE:BF,eth1,eth1,10.128.1.2,4,HundredGigE0/0/0/5
Node2-nic1-p1,p1,Node2,net2,100,auto,,02:34:83:E8:AE:A3,eth1,eth1,10.128.129.2,6,HundredGigE0/0/0/5



Time to print interfaces 313 seconds


## Add the Listener to the Slice
Add a listener node to the slice which mirrors the traffic received on `Node1`

In [12]:
slice = fablib.get_slice(slice_name)
node1 = slice.get_node(node1_name)

mirror_port_name = node1.get_interfaces()[0].get_peer_port_name()
mirror_port_vlan = node1.get_interfaces()[0].get_peer_port_vlan()

listener_node = slice.add_node(name=listener_node_name, site=site1)
# the first (index 0) interface will be connected into the switch to receive mirror traffic
listener_interface = listener_node.add_component(model='NIC_Basic', name='pmnic').get_interfaces()[0]

# port mirroring is a network service of a special kind
# it mirrors one or both directions of traffic ('rx', 'tx' or 'both') of a port that we identified in
# Traffic Generator Topology into a port of a card we allocated in this slice (listener_interface)
# NOTE: if you select 'both' directions that results in potentially 200Gbps of traffic, which
# of course is impossible to fit into a single 100Gbps port of a ConnectX_6 card - be mindful of the
# data rates.
pmnet = slice.add_port_mirror_service(name=listener_pm_service, 
                                      mirror_interface_name=mirror_port_name,
                                      mirror_interface_vlan=mirror_port_vlan,
                                      receive_interface=listener_interface,
                                      mirror_direction = listener_direction)


#Submit Slice Request
slice.submit();


Retry: 77, Time: 1821 sec


ID,b3c38986-639c-42e3-860a-35d3fc3fa594
Name,Traffic Generator_Listener Slice
Lease Expiration (UTC),2024-07-31 21:19:22 +0000
Lease Start (UTC),2024-07-30 21:19:22 +0000
Project ID,b9847fa1-13ef-49f9-9e07-ae6ad06cda3f
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
2e640c4f-d444-41a8-85e8-ace38ce17015,Node1,2,8,10,default_rocky_8,qcow2,renc-w1.fabric-testbed.net,RENC,rocky,152.54.15.39,Active,,ssh -i {{ _self_.private_ssh_key_file }} -F /home/fabric/work/fabric_config/ssh_config {{ _self_.username }}@{{ _self_.management_ip }},/home/fabric/work/fabric_config/id_rsa.pub,/home/fabric/work/fabric_config/id_rsa
8360385c-c21c-4f34-a27d-3c77d6aac2f1,Node2,2,8,10,default_rocky_8,qcow2,uky-w1.fabric-testbed.net,UKY,rocky,128.163.179.49,Active,,ssh -i {{ _self_.private_ssh_key_file }} -F /home/fabric/work/fabric_config/ssh_config {{ _self_.username }}@{{ _self_.management_ip }},/home/fabric/work/fabric_config/id_rsa.pub,/home/fabric/work/fabric_config/id_rsa
None,listener_node,None,None,None,default_rocky_8,qcow2,None,RENC,rocky,,None,,,/home/fabric/work/fabric_config/id_rsa.pub,/home/fabric/work/fabric_config/id_rsa


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
505b620d-b7d1-4b47-ba4e-15d26ae5787c,net1,L3,FABNetv4,RENC,10.128.1.0/24,10.128.1.1,Active,
acecd0b0-5b68-409c-9579-f9674becb657,net2,L3,FABNetv4,UKY,10.128.129.0/24,10.128.129.1,Active,
None,pmservice,L2,PortMirror,None,pmservice.subnet,pmservice.gateway,None,


Exception: Timeout 1800 sec exceeded in Jupyter wait

### Test the slice

Let's make sure the two nodes i.e. `Node1` and `Node2` can communicate as expected.

In [ ]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           

node2_addr = node2.get_interface(network_name=network2_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')

### Query the Listener slice

This cell is useful if you are coming back to the experiment after a while. 

In [ ]:
pmslice = fablib.get_slice(listener_slice_name)

listener_node = pmslice.get_node(name=listener_node_name)   

pmslice.list_nodes()
pmslice.list_networks()

## Deal with IPv4/IPv6 and install missing packages

On this slice we will need to install tcpdump so we can watch the mirrored traffic. This means we need to update DNS to use NAT64 in case the site is IPv6.

In [ ]:
from ipaddress import ip_address, IPv6Address    

isipv6_site = False
# If the node is an IPv6 Node then configure NAT64
if type(ip_address(listener_node.get_management_ip())) is IPv6Address:
    isipv6_site = True
    print(f'Node {listener_node.get_name()} has an IPv6 management address, will update DNS configuration')
    
# this code will be executed if the node uses an IPv6 site. See the notebook 
# 'Access non-IPv6 services (i.e. GitHub) from IPv6 FABRIC nodes' for more details

if isipv6_site:
    listener_node.upload_file('../accessing_ipv4_services_from_ipv6_nodes/nat64.sh', 'nat64.sh')
    stdout, stderr = listener_node.execute(f'chmod +x nat64.sh && ./nat64.sh')
    print(f'Uploaded and executed NAT64 DNS setup script to node {listener_node.get_name()}')
    

And install tcpdump package

In [ ]:
command = 'sudo dnf install -y tcpdump'

stdout, stderr = listener_node.execute(command)

## Run the experiment

As a trivial experiment we start a `ping` between the two nodes in the Traffic Generator Slice and see if we can see it in the Lister Slice.

In [ ]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           
listener_node = pmslice.get_node(name=listener_node_name)

node2_addr = node2.get_interface(network_name=network2_name).get_ip_addr()
listener_node_intf_name = listener_node.get_interface(network_name=listener_pm_service).get_device_name()

print(f'Will run tcpdump on {listener_node.get_name()} interface {listener_node_intf_name}, listening to ping in Traffic Generator Slice')

# run everything for 10 seconds
traffic_command = f'timeout 10s ping -c 100 {node2_addr}'
# look at libpcap documentation for more info on how to filter packets
listen_command = f'sudo timeout 10s tcpdump -i {listener_node_intf_name} icmp'

# start traffic generation in the background
node2_thread = node1.execute_thread(traffic_command, output_file='node1_ping.log')
# start tcpdump in the foreground
listener_node_thread = listener_node.execute(listen_command)

# check for errors
stdout, stderr = node2_thread.result()
if stderr and len(stderr):
    print(f'Error output from Traffic Generator Slice: {stderr}')
    
print(f'Done')

## Delete both slices

In [ ]:
try:
    fablib.delete_slice(slice_name)
except Exception as e:
    print(e)

print('Done')